In [232]:
# Importar Librerías
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import keras_tuner as kt


In [233]:
#Cargar Archivo
df1 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cordoba1")
df2 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira2")
df3 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira3")
df4 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira4")
df5 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Antioquia5")
df6 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Antioquia6")
df7 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Atlantico7")
df8 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Atlantico8")
df9 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Magdalena9")
df10 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Magdalena10")
df11 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cesar11")
df12 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cesar12")
df13 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Bolivar13")
df14 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Bolivar14")
df15 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Choco15")

# Lista Df
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15]

# Unirlos en uno solo
df = pd.concat(dfs, ignore_index=True)

In [234]:
#Explorar Archivo
df.head()

,Hoja,Station Number,Año,Día,Irradiacion,Min Temp,Max Temp,Temp Promedio,Vapor,Wind Speed,Precipitacion,Departamento,Irrigacion,Produccion,Toneladas por hectaria
0,2000,1,2000,183.5,4593.10,23.34,32.93,28.14,38.22,1.05,5.83,1,1,5969.732058,59.697321
1,2001,1,2001,183.0,4591.66,23.84,33.27,28.56,39.24,1.09,5.84,1,1,5625.796938,56.257969
2,2002,1,2002,183.0,4591.66,24.13,33.74,28.94,40.13,0.94,5.84,1,1,4309.850302,43.098503
3,2003,1,2003,183.0,4591.66,24.08,33.28,28.68,39.50,1.11,5.84,1,1,4121.129416,41.211294
4,2004,1,2004,183.5,4593.10,24.40,33.39,28.90,40.11,1.05,5.83,1,1,4565.883797,45.658838


In [235]:
#Emplear las varibales significativas
df = df[["Irradiacion","Max Temp","Precipitacion","Vapor","Min Temp","Departamento","Toneladas por hectaria"]]

In [236]:
#Dividir el dataset en 80% entrenamiento y 20% prueba
train = df.sample(frac=0.8, random_state=100)
test = df.drop(train.index)
test.head()

,Irradiacion,Max Temp,Precipitacion,Vapor,Min Temp,Departamento,Toneladas por hectaria
0,4593.10,32.93,5.83,38.22,23.34,1,59.697321
2,4591.66,33.74,5.84,40.13,24.13,1,43.098503
4,4593.10,33.39,5.83,40.11,24.40,1,45.658838
13,4591.66,33.49,5.84,40.40,24.61,1,45.378550
14,4591.66,33.87,5.84,40.02,23.99,1,47.003314


In [237]:
#Separar características (X) y variable objetivo (y)
train_X = train.copy()
test_X = test.copy()
# Asegurar que las características sean de tipo float para evitar errores
train_X = train_X.astype(float)
test_X = test_X.astype(float)

In [238]:
## Extraer columna objetivo
train_y = train_X.pop('Toneladas por hectaria')
test_y = test_X.pop('Toneladas por hectaria')

In [239]:
#Construir modelo
#Capa de normalizacion
norm = tf.keras.layers.Normalization()
# Adaptar la capa de normalización a los datos de entrenamiento
norm.adapt(np.array(train_X))

MODELOS

In [ ]:
#Modelos Capas
def construir_modelo(norm_layer, capas, input_shape):
    modelo = tf.keras.Sequential([norm_layer])
    for i, neuronas in enumerate(capas):
        if i == 0:
            modelo.add(tf.keras.layers.Dense(neuronas, activation='relu', input_shape=(input_shape,)))
        else:
            modelo.add(tf.keras.layers.Dense(neuronas, activation='relu'))
    modelo.add(tf.keras.layers.Dense(1))  # Capa de salida
    return modelo

def entrenar_evaluar_modelo(nombre_modelo, capas, resultados_list):
    modelo = construir_modelo(norm, capas, train_X.shape[1])
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mean_absolute_error',
        metrics=['mae']
    )
    history = modelo.fit(
        train_X,
        train_y,
        epochs=100,
        validation_split=0.2,
        verbose=1
    )
    y_pred = modelo.predict(test_X).flatten()
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)

    resultados_list.append({
        'Modelo': nombre_modelo,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Modelo_Objeto': modelo  # Guardamos el objeto modelo también
    })

# Lista para resultados
resultados_modelos = []

# Probar modelos con diferentes configuraciones de capas
configuraciones = {
    'Modelo1': [64],
    'Modelo2': [128, 64],
    'Modelo3': [128, 64, 32],
    'Modelo4': [256, 128, 64, 32]
}

for nombre, capas in configuraciones.items():
    entrenar_evaluar_modelo(nombre, capas, resultados_modelos)

# Crear DataFrame
resultados_df = pd.DataFrame(resultados_modelos).drop(columns=['Modelo_Objeto'])
print(resultados_df)

# Seleccionar el mejor modelo por MAE (puedes cambiar a 'RMSE' o 'R2' si prefieres)
mejor_resultado = min(resultados_modelos, key=lambda x: x['MAE'])
print(f"\nMejor modelo: {mejor_resultado['Modelo']} con MAE = {mejor_resultado['MAE']:.4f}")

# Acceder al objeto del mejor modelo
mejor_modelo = mejor_resultado['Modelo_Objeto']

Epoch 1/100


c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 28.8986 - mae: 28.8986 - val_loss: 29.9749 - val_mae: 29.9749
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.8957 - mae: 29.8957 - val_loss: 30.1569 - val_mae: 30.1569
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.8215 - mae: 29.8215 - val_loss: 30.3785 - val_mae: 30.3785
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.9626 - mae: 28.9626 - val_loss: 30.6694 - val_mae: 30.6694
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.6398 - mae: 28.6398 - val_loss: 31.0489 - val_mae: 31.0489
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.6740 - mae: 29.6740 - val_loss: 31.6098 - val_mae: 31.6098
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.0682 - mae: 27.0682 - val_loss: 32.0415 - val_mae: 32.0415
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.3900 - mae: 29.3900 - val_loss: 32.8317 - val_mae: 32.8317
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 31.5568 - mae: 31.5568 - val_loss: 30.2860 - val_mae: 30.2860
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.8906 - mae: 30.8906 - val_loss: 31.4548 - val_mae: 31.4548
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.1841 - mae: 29.1841 - val_loss: 33.2598 - val_mae: 33.2598
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.5349 - mae: 29.5349 - val_loss: 32.4842 - val_mae: 32.4842
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.3084 - mae: 27.3084 - val_loss: 33.7916 - val_mae: 33.7916
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.5473 - mae: 29.5473 - val_loss: 34.4675 - val_mae: 34.4675
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 27.0925 - mae: 27.0925 - val_loss: 34.4351 - val_mae: 34.4351
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.0351 - mae: 27.0351 - val_loss: 34.3491 - val_mae: 34.3491
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 30.3571 - mae: 30.3571 - val_loss: 31.1139 - val_mae: 31.1139
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.4165 - mae: 28.4165 - val_loss: 32.8470 - val_mae: 32.8470
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.9895 - mae: 28.9895 - val_loss: 33.8738 - val_mae: 33.8738
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.9386 - mae: 29.9386 - val_loss: 35.2278 - val_mae: 35.2278
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 26.5691 - mae: 26.5691 - val_loss: 35.2343 - val_mae: 35.2343
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.3833 - mae: 28.3833 - val_loss: 34.3185 - val_mae: 34.3185
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.4044 - mae: 29.4044 - val_loss: 34.9101 - val_mae: 34.9101
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.1891 - mae: 28.1891 - val_loss: 37.0089 - val_mae: 37.0089
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 29.5648 - mae: 29.5648 - val_loss: 32.4055 - val_mae: 32.4055
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 25.9444 - mae: 25.9444 - val_loss: 32.5759 - val_mae: 32.5759
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.1435 - mae: 29.1435 - val_loss: 34.1939 - val_mae: 34.1939
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.1724 - mae: 28.1724 - val_loss: 34.7182 - val_mae: 34.7182
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 26.2753 - mae: 26.2753 - val_loss: 33.5736 - val_mae: 33.5736
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.9033 - mae: 29.9033 - val_loss: 35.4214 - val_mae: 35.4214
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.1258 - mae: 28.1258 - val_loss: 34.0885 - val_mae: 34.0885
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.2577 - mae: 28.2577 - val_loss: 33.0161 - val_mae: 33.0161
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

In [ ]:
#Modelos Activación
# Obtener la mejor arquitectura de capas del punto anterior
mejores_capas = configuraciones[mejor_resultado['Modelo']]

# Lista de funciones de activación a probar
activaciones = ['relu', 'tanh', 'sigmoid', 'elu', 'selu']

# Lista para nuevos resultados por activación
resultados_activaciones = []

def construir_modelo_activacion(norm_layer, capas, input_shape, activacion):
    modelo = tf.keras.Sequential([norm_layer])
    for i, neuronas in enumerate(capas):
        if i == 0:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion, input_shape=(input_shape,)))
        else:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion))
    modelo.add(tf.keras.layers.Dense(1))  # Capa de salida
    return modelo

def entrenar_evaluar_modelo_activacion(nombre_modelo, capas, activacion, resultados_list):
    modelo = construir_modelo_activacion(norm, capas, train_X.shape[1], activacion)
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mean_absolute_error',
        metrics=['mae']
    )
    history = modelo.fit(
        train_X,
        train_y,
        epochs=100,
        validation_split=0.2,
        verbose=1
    )
    y_pred = modelo.predict(test_X).flatten()
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)

    resultados_list.append({
        'Modelo': f"{nombre_modelo}_{activacion}",
        'Activacion': activacion,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Modelo_Objeto': modelo
    })

# Probar activaciones con las mejores capas
for activacion in activaciones:
    entrenar_evaluar_modelo_activacion("MejorModelo", mejores_capas, activacion, resultados_activaciones)

# Crear DataFrame
resultados_activaciones_df = pd.DataFrame(resultados_activaciones).drop(columns=['Modelo_Objeto'])
print(resultados_activaciones_df)

# Elegir el mejor modelo por activación
mejor_activacion_resultado = min(resultados_activaciones, key=lambda x: x['MAE'])
print(f"\nMejor modelo por activación: {mejor_activacion_resultado['Modelo']} con MAE = {mejor_activacion_resultado['MAE']:.4f}")

# Modelo final
modelo_final = mejor_activacion_resultado['Modelo_Objeto']

#Modelos
# Acceder al objeto del mejor modelo
mejor_modelo = mejor_resultado['Modelo_Objeto']
mejores=[mejor_activacion_resultado['Modelo_Objeto'],mejor_resultado['Modelo_Objeto']]


Epoch 1/100


c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 31.1177 - mae: 31.1177 - val_loss: 29.9311 - val_mae: 29.9311
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.6981 - mae: 29.6981 - val_loss: 30.0977 - val_mae: 30.0977
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.2250 - mae: 28.2250 - val_loss: 30.3175 - val_mae: 30.3175
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.6334 - mae: 30.6334 - val_loss: 30.5478 - val_mae: 30.5478
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.7225 - mae: 28.7225 - val_loss: 30.9220 - val_mae: 30.9220
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.9431 - mae: 29.9431 - val_loss: 31.4147 - val_mae: 31.4147
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.7977 - mae: 27.7977 - val_loss: 31.8388 - val_mae: 31.8388
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.5050 - mae: 29.5050 - val_loss: 32.6222 - val_mae: 32.6222
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 29.8397 - mae: 29.8397 - val_loss: 29.9217 - val_mae: 29.9217
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.8177 - mae: 30.8177 - val_loss: 30.0531 - val_mae: 30.0531
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 31.6395 - mae: 31.6395 - val_loss: 30.2098 - val_mae: 30.2098
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.3705 - mae: 30.3705 - val_loss: 30.3634 - val_mae: 30.3634
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 31.2529 - mae: 31.2529 - val_loss: 30.6300 - val_mae: 30.6300
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.7003 - mae: 29.7003 - val_loss: 30.8353 - val_mae: 30.8353
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.4866 - mae: 29.4866 - val_loss: 31.1701 - val_mae: 31.1701
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.1681 - mae: 29.1681 - val_loss: 31.6562 - val_mae: 31.6562
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_20                │ (None, 6)              │            13 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_129 (Dense)               │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_130 (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,554 (6.08 KB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 13 (56.00 B)

 Optimizer params: 1,028 (4.02 KB)

In [249]:
# --- FUNCIONES ---
def construir_modelo(norm_layer, capas, input_shape, activacion='relu'):
    modelo = tf.keras.Sequential([norm_layer])
    for i, neuronas in enumerate(capas):
        if i == 0:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion, input_shape=(input_shape,)))
        else:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion))
    modelo.add(tf.keras.layers.Dense(1))  # Capa de salida
    return modelo

def entrenar_evaluar_modelo(nombre_modelo, capas, resultados_list, activacion='relu'):
    modelo = construir_modelo(norm, capas, train_X.shape[1], activacion)
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mean_absolute_error',
        metrics=['mae']
    )
    history = modelo.fit(
        train_X,
        train_y,
        epochs=100,
        validation_split=0.2,
        verbose=1
    )
    y_pred = modelo.predict(test_X).flatten()
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)

    resultados_list.append({
        'Modelo': nombre_modelo,
        'Capas': str(capas),
        'Activacion': activacion,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Modelo_Objeto': modelo
    })

# --- PRUEBA CON DIFERENTES CAPAS ---
resultados_modelos = []

configuraciones = {
    'Modelo1': [64],
    'Modelo2': [128, 64],
    'Modelo3': [128, 64, 32],
    'Modelo4': [256, 128, 64, 32]
}

for nombre, capas in configuraciones.items():
    entrenar_evaluar_modelo(nombre, capas, resultados_modelos)

# --- MEJOR MODELO SEGÚN MAE ---
mejor_resultado = min(resultados_modelos, key=lambda x: x['MAE'])
mejores_capas = configuraciones[mejor_resultado['Modelo']]

# --- PRUEBA DE ACTIVACIONES CON LAS MEJORES CAPAS ---
activaciones = ['relu', 'tanh', 'sigmoid', 'elu', 'selu']

for activacion in activaciones:
    entrenar_evaluar_modelo(f"MejorModelo_{activacion}", mejores_capas, resultados_modelos, activacion)

# --- CREAR DATAFRAME FINAL ---
resultados_df = pd.DataFrame(resultados_modelos).drop(columns=['Modelo_Objeto'])
print(resultados_df)

# --- ELEGIR EL MEJOR MODELO GLOBAL ---
mejor_modelo_general = min(resultados_modelos, key=lambda x: x['MAE'])
print(f"\n🏆 Mejor modelo total: {mejor_modelo_general['Modelo']} | MAE = {mejor_modelo_general['MAE']:.4f} | Activación: {mejor_modelo_general['Activacion']}")

# --- ACCEDER A LOS OBJETOS DE LOS MEJORES MODELOS ---
modelo_mejor_capas = mejor_resultado['Modelo_Objeto']
modelo_mejor_activacion = mejor_modelo_general['Modelo_Objeto']
mejores = [modelo_mejor_activacion, modelo_mejor_capas]


Epoch 1/100


c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 29.9263 - mae: 29.9263 - val_loss: 29.9807 - val_mae: 29.9807
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.4134 - mae: 29.4134 - val_loss: 30.1412 - val_mae: 30.1412
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 30.9775 - mae: 30.9775 - val_loss: 30.3147 - val_mae: 30.3147
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 31.9808 - mae: 31.9808 - val_loss: 30.6219 - val_mae: 30.6219
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.6361 - mae: 30.6361 - val_loss: 31.0322 - val_mae: 31.0322
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 30.8531 - mae: 30.8531 - val_loss: 31.6242 - val_mae: 31.6242
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27.5097 - mae: 27.5097 - val_loss: 32.2370 - val_mae: 32.2370
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 26.2982 - mae: 26.2982 - val_loss: 32.6556 - val_mae: 32.6556
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 31.1057 - mae: 31.1057 - val_loss: 30.6112 - val_mae: 30.6112
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.3635 - mae: 28.3635 - val_loss: 32.2123 - val_mae: 32.2123
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.9706 - mae: 27.9706 - val_loss: 32.8152 - val_mae: 32.8152
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.1841 - mae: 28.1841 - val_loss: 33.4463 - val_mae: 33.4463
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.2686 - mae: 30.2686 - val_loss: 34.0467 - val_mae: 34.0467
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.0425 - mae: 30.0425 - val_loss: 34.1539 - val_mae: 34.1539
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.6127 - mae: 27.6127 - val_loss: 34.7068 - val_mae: 34.7068
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.5143 - mae: 28.5143 - val_loss: 36.0420 - val_mae: 36.0420
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 27.8608 - mae: 27.8608 - val_loss: 29.9723 - val_mae: 29.9723
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.7808 - mae: 30.7808 - val_loss: 30.1487 - val_mae: 30.1487
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.0617 - mae: 31.0617 - val_loss: 30.2660 - val_mae: 30.2660
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 29.5424 - mae: 29.5424 - val_loss: 30.3510 - val_mae: 30.3510
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.4512 - mae: 29.4512 - val_loss: 30.4849 - val_mae: 30.4849
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.3255 - mae: 31.3255 - val_loss: 30.6661 - val_mae: 30.6661
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.8383 - mae: 30.8383 - val_loss: 30.8100 - val_mae: 30.8100
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.6453 - mae: 30.6453 - val_loss: 31.0391 - val_mae: 31.0391
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 30.7022 - mae: 30.7022 - val_loss: 29.9769 - val_mae: 29.9769
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.2272 - mae: 29.2272 - val_loss: 30.0185 - val_mae: 30.0185
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.1133 - mae: 30.1133 - val_loss: 30.2666 - val_mae: 30.2666
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.4278 - mae: 29.4278 - val_loss: 30.3075 - val_mae: 30.3075
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.3066 - mae: 29.3066 - val_loss: 30.5771 - val_mae: 30.5771
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.5799 - mae: 29.5799 - val_loss: 31.1300 - val_mae: 31.1300
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.1186 - mae: 30.1186 - val_loss: 31.4484 - val_mae: 31.4484
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.1693 - mae: 30.1693 - val_loss: 32.1331 - val_mae: 32.1331
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 29.3637 - mae: 29.3637 - val_loss: 29.9710 - val_mae: 29.9710
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.6375 - mae: 29.6375 - val_loss: 30.3171 - val_mae: 30.3171
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.4630 - mae: 31.4630 - val_loss: 30.3595 - val_mae: 30.3595
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.8258 - mae: 30.8258 - val_loss: 30.5973 - val_mae: 30.5973
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.1353 - mae: 31.1353 - val_loss: 30.8160 - val_mae: 30.8160
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.2792 - mae: 30.2792 - val_loss: 31.3593 - val_mae: 31.3593
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29.2642 - mae: 29.2642 - val_loss: 31.6704 - val_mae: 31.6704
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.7834 - mae: 27.7834 - val_loss: 32.3672 - val_mae: 32.3672
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5